In [3]:
#! /usr/bin/env python3
import multiCMD
import TSVZ
import inotify_simple
import Tee_Logger
import argparse
import xxhash

from collections import deque
import time
from collections import namedtuple
from collections import OrderedDict
import os
import threading
import sys
import signal
import resource
import datetime
import unicodedata
import re
from shutil import copystat

# for the lib wrapping aroung inotify, I tried
# wtr-wather, watchdog, watchfiles
# None of them seem to work well with the most basic tests
# I need it to work with:
# 1. able to notice changes recursively
# 2. able to notice changes in the folders
# 3. do not incorrectly group move events
# 4. recognize mkdir -p properly
# 5. notice umount events and notify properly
# 6. notice mount events and notify properly
# 7. compatible with symlink events 


__version__ = 0.01

## WIP

# @dataclass
# class ChangeEvent:
# 	monotonic_time:float
# 	# type can be file or dir.
# 	type:str
# 	# if type is dir, event can be create 
# 	#  ( need to recursively copy everything as inotifywait does not caputre mkdir -p properly for all sub folders),
# 	#  ,delete, attrib, move
# 	# if type is file, event can be delete, change, attrib, move
# 	# move detection will be handled by monitor if moved_from moved_to events were captured. 
# 	# it will use an tsv storing all the path inode number and hash to detect move.
# 	event:str
# 	path:str
# 	moved_from:str = None


# if type is dir, event can be create 
#  ( need to recursively copy everything as inotifywait does not caputre mkdir -p properly for all sub folders),
#  ,delete, attrib, move
# if type is file, event can be delete, change, attrib, move
# move detection will be handled by monitor if moved_from moved_to events were captured. 
# it will use an tsv storing all the path inode number and hash to detect move.
CHANGED_EVENT_HEADER = ['monotonic_time', 'is_dir', 'event', 'path','moved_from']
BACKUP_ENTRY_VALUES_HEADER = ['iso_time','event','source_path']
BACKUP_JOURNAL_HEADER = ['at_path','iso_time','event','source_path']
REVERB_RULE_TSV_HEADER = ['Job Name (Unique Key)', 'Path Monitoring', 'Monitoring File System Signiture', 
		'Minium snapshot time', 'Vault Path', 'Keep 1 Compelete Backup', 
		'Only Sync Attributes (permissions)', 'Keep N versions', 'Backup Size Limit', 
		'Vault File System Signiture']
REVERB_RULE_HEADER = ['job_name', 'mon_path', 'mon_fs_signiture', 
		'min_shapshot_time', 'vault_path', 'keep_one_complete_backup', 
		'only_sync_attributes', 'keep_n_versions', 'backup_size_limit',
		'vault_fs_signiture']
VAULT_ENTRY_HEADER = ['version_number','path','timestamp','size','inode']
VAULT_TIMESTAMP_FORMAT = '%Y-%m-%d_%H-%M-%S_%z'

ChangeEvent = namedtuple('ChangeEvent', CHANGED_EVENT_HEADER)
BackupEntryValues = namedtuple('BackupEntryValues', BACKUP_ENTRY_VALUES_HEADER)
ReverbRule = namedtuple('ReverbRule', REVERB_RULE_HEADER)
VaultEntry = namedtuple('VaultEntry', VAULT_ENTRY_HEADER)

# by defualt, set the max backup threads to 2* the number of cores
BACKUP_SEMAPHORE = threading.Semaphore(2*os.cpu_count())

DEFAULT_SNAPSHOT_DELAY = 60
DEFAULT_KEEP_ONE_COMPLETE_BACKUP = False
DEFAULT_ONLY_SYNC_ATTRIBUTES = False
DEFAULT_KEEP_N_VERSIONS = 30
DEFAULT_BACKUP_SIZE_LIMIT = '5%'

COOKIE_DICT_MAX_SIZE = 16384
COOKIE_VALUE = namedtuple('COOKIE_VALUE',['wd','path','isDir'])

GREEN_LIGHT = threading.Event()
GREEN_LIGHT.set()

DEBUG = True

tl = Tee_Logger.teeLogger(systemLogFileDir='/dev/null', programName='reverberator', compressLogAfterMonths=0, deleteLogAfterYears=0, suppressPrintout=False,noLog=True)

DEBUG = True
os.chdir('/tmp')

In [4]:
from collections import namedtuple
# if type is dir, event can be create 
#  ( need to recursively copy everything as inotifywait does not caputre mkdir -p properly for all sub folders),
#  ,delete, attrib, move
# if type is file, event can be delete, change, attrib, move
# move detection will be handled by monitor if moved_from moved_to events were captured. 
# it will use an tsv storing all the path inode number and hash to detect move.
CHANGED_EVENT_HEADER = ['monotonic_time', 'is_dir', 'event', 'path','moved_from']
BACKUP_ENTRY_VALUES_HEADER = ['iso_time','event','source_path']

ChangeEvent = namedtuple('ChangeEvent', CHANGED_EVENT_HEADER)
BackupEntryValues = namedtuple('BackupEntryValues', BACKUP_ENTRY_VALUES_HEADER)

DEFAULT_SNAPSHOT_DELAY = 5



In [5]:
if not os.path.exists('/tmp/inotify_test'):
	os.mkdir('/tmp/inotify_test')

flags = inotify_simple.flags
watch_flags = flags.MODIFY | flags.ATTRIB | flags.MOVED_FROM | flags.MOVED_TO | flags.CREATE | flags.DELETE | flags.DELETE_SELF | flags.MOVE_SELF | flags.UNMOUNT | flags.Q_OVERFLOW


with inotify_simple.INotify() as inotify:
	wd = inotify.add_watch('/tmp/inotify_test', watch_flags)
	# Now create, delete and modify some files in the directory being monitored:
	os.chdir('/tmp/inotify_test')

	os.system('mkdir foo')
	wd= inotify.add_watch('/tmp/inotify_test/foo', watch_flags)
	os.system('echo hello > test.txt')
	os.system('echo world >> test.txt')
	os.system('mv test.txt test2.txt')
	os.system('mv test2.txt ../outside.txt')
	os.system('echo hello > ../outside2.txt')
	os.system('mv ../outside2.txt test2.txt')
	os.system('mv test2.txt ../outside3.txt')
	os.system('mv ../outside3.txt test3.txt')
	os.system('rm test3.txt')
	os.system('echo hi >> foo/test_rec.txt')
	os.system('mv foo bar')
	os.system('touch bar/test.txt')
	os.system('echo hello > bar/test.txt')
	os.system('mv bar/test.txt bar/test2.txt')
	# os.system('sudo mount -t tmpfs tmpfs bar')
	# os.system('sudo umount bar')
	os.system('touch bar/test_temp.txt')
	os.system('touch bar')
	os.system('mkdir -p test/testrecursive')
	inotify.rm_watch(wd)
	os.chdir('/tmp')
	# DELETE_SELF on the original directory. # Also generates an IGNORED event
	# indicating the watch was removed.
	os.system('mv inotify_test inotify_test_2')
	os.system('rm -rf inotify_test_2')
	os.system('mkdir inotify_test')
	events = inotify.read()

In [6]:
events[0]

Event(wd=1, mask=1073742080, cookie=0, name='foo')

In [7]:
# decode the masks 
decodedEvents = [[event.wd, flags.from_mask(event.mask),event.cookie, event.name] for event in events]
print(TSVZ.pretty_format_table(decodedEvents,header=['wd', 'mask','cookie','name']))

wd | mask                                                | cookie | name         
---+-----------------------------------------------------+--------+--------------
1  | [<flags.CREATE: 256>, <flags.ISDIR: 1073741824>]    | 0      | foo          
1  | [<flags.CREATE: 256>]                               | 0      | test.txt     
1  | [<flags.MODIFY: 2>]                                 | 0      | test.txt     
1  | [<flags.MOVED_FROM: 64>]                            | 5883   | test.txt     
1  | [<flags.MOVED_TO: 128>]                             | 5883   | test2.txt    
1  | [<flags.MOVED_FROM: 64>]                            | 5884   | test2.txt    
1  | [<flags.MOVED_TO: 128>]                             | 5885   | test2.txt    
1  | [<flags.MOVED_FROM: 64>]                            | 5886   | test2.txt    
1  | [<flags.MOVED_TO: 128>]                             | 5887   | test3.txt    
1  | [<flags.DELETE: 512>]                               | 0      | test3.txt    
2  | [<flags.CRE

In [8]:
from collections import OrderedDict
cookieDic = OrderedDict()
COOKIE_DICT_MAX_SIZE = 16384
COOKIE_VALUE = namedtuple('COOKIE_VALUE',['wd','path','isDir'])

In [9]:
from collections import deque
to_process = deque()

In [10]:
import time
import multiCMD

In [11]:
import threading
GREEN_LIGHT = threading.Event()
GREEN_LIGHT.set()

In [12]:
import Tee_Logger
tl = Tee_Logger.teeLogger(systemLogFileDir='/dev/null', programName='reverberator', compressLogAfterMonths=0, deleteLogAfterYears=0, suppressPrintout=False,noLog=True)

In [13]:
def get_all_folders(path):
    folders = []
    try:
        with os.scandir(path) as entries:
            for entry in entries:
                if entry.is_dir(follow_symlinks=False):
                    folders.append(entry.path)
                    folders.extend(get_all_folders(entry.path))  # Recursive call
    except PermissionError:
        pass  # Skip folders without permission
    return folders

In [14]:
for i in range(10):
	folders = get_all_folders(os.path.expanduser('.'))
print(len(folders))
#folders

26


In [15]:
rtn = multiCMD.run_command(['sysctl','fs.inotify.max_queued_events'],timeout=5,quiet=True)

In [16]:
resource.getrlimit(resource.RLIMIT_NOFILE)[0]

524288

In [17]:
test = {'1':'test'}
test.popitem()

('1', 'test')

In [18]:
# security limits
class inotify_resource_manager:
	def __init__(self):
		self.current_user_instances = 0
		self.current_user_watches = 0
		self.can_set_max_queued_events = True
		self.can_set_max_user_instances = True
		self.can_set_max_user_watches = True
		self.can_set_max_no_files = True
		self.can_set_max_no_files_user = True
		self.debug = False
		self.max_queued_events = self.getSysctlValues('fs.inotify.max_queued_events')
		self.max_user_instances = self.getSysctlValues('fs.inotify.max_user_instances')
		self.max_user_watches = self.getSysctlValues('fs.inotify.max_user_watches')
		self.max_no_files = self.getSysctlValues('fs.file-max')
		try:
			soft , hard = resource.getrlimit(resource.RLIMIT_NOFILE)
			if soft < hard:
				resource.setrlimit(resource.RLIMIT_NOFILE, (hard, hard))
			self.max_no_files_user = hard
		except:
			self.max_no_files_user = 0

	def __iter__(self):
		for key in self.__dict__.keys():
			yield key, getattr(self,key)

	def __str__(self):
		# do not include the can_set in the string representation
		return str(dict((k, v) for k, v in self.__dict__.items() if not k.startswith('can_set_')))
	
	def __repr__(self):
		return str(dict(self))

	def getSysctlValues(self,field):
		try:
			rtn = multiCMD.run_command(['sysctl',field],timeout=1,quiet=not self.debug)
			if rtn and rtn[0]:
				return int(rtn[0].split('=')[-1].strip())
		except:
			import traceback
			tl.error(traceback.format_exc())
			tl.error(f'Failed to get {field} value')
		return 0
	
	def setSysctlValues(self,field,value,value_name):
		try:
			if not getattr(self,f'can_set_{value_name}'):
				return False
			if value is ...:
				value = getattr(self,value_name)
			# check if the current value is less
			rtn = self.getSysctlValues(field)
			if rtn and rtn > value:
				tl.info(f'Current {field} is already higher than {value}')
				setattr(self,value_name,rtn)
				return False
			rc = multiCMD.run_command(['sudo','sysctl',f'{field}={value}'],timeout=1,quiet=not self.debug,return_code_only=True)
			if self.debug:
				tl.teeprint(f'sudo sysctl {field}={value} return code = {rc}')
			if rc:
				if self.debug:
					tl.teeprint(f'Failed to set {field} to {value}')
				else:
					tl.info(f'Failed to set {field} to {value}')
				setattr(self,f'can_set_{value_name}',False)
				if rtn:
					setattr(self,value_name,rtn)
				return False
			else:
				tl.teeok(f'Increased inotify {field} to {value}')
				setattr(self,value_name,value)
				return True
		except:
			try:
				import traceback
				tl.error(traceback.format_exc())
				tl.teeerror(f'Failed to set {field} = {value} for {value_name}')
				setattr(self,f'can_set_{value_name}',False)
				rtn = self.getSysctlValues(field)
				if rtn:
					setattr(self,value_name,rtn)
				else:
					setattr(self,value_name,-1)
			except:
				tl.teeerror(f'Failed to set {field} value')
		return False

	def increaseInotifyMaxQueueEvents(self,max_events:int = ...):
		if not self.can_set_max_queued_events:
			return False
		return self.setSysctlValues('fs.inotify.max_queued_events',max_events,'max_queued_events')

	def increaseInotifyMaxUserInstances(self,max_instances:int = ...):
		if not self.can_set_max_user_instances:
			return False
		return self.setSysctlValues('fs.inotify.max_user_instances',max_instances,'max_user_instances')

	def increaseInotifyMaxUserWatches(self,max_watches:int = ...):
		if not self.can_set_max_user_watches:
			return False
		rtn = self.setSysctlValues('fs.inotify.max_user_watches',max_watches,'max_user_watches')
		rtn2 = self.increaseNoFiles(max_watches)
		return rtn and rtn2

	def increaseNoFiles(self,nofile:int = ...):
		if not self.can_set_max_no_files_user:
			return False
		if nofile is ...:
			nofile = self.max_no_files
		# attempt to increase the open file limit
		try:
			# check if the current value is less
			if resource.getrlimit(resource.RLIMIT_NOFILE)[0] > nofile:
				if self.debug:
					tl.teeprint(f'Current open file limit is already higher than {nofile}')
				else:
					tl.info(f'Current open file limit is already higher than {nofile}')
				self.max_no_files = resource.getrlimit(resource.RLIMIT_NOFILE)[0]
				return False
			rtn = self.__increaseSystemNoFiles(nofile)
			resource.setrlimit(resource.RLIMIT_NOFILE, (nofile, nofile))
			tl.teeok(f'Increased open file limit to {nofile}')
			self.max_no_files = nofile
			return rtn
		except:
			import traceback
			tl.error(traceback.format_exc())
			tl.teeerror(f'Failed to increase open file limit to {nofile}')
			self.can_set_max_no_files_user = False
			return False

	def __increaseSystemNoFiles(self,nofile:int):
		if not self.can_set_max_no_files:
			return False
		return self.setSysctlValues('fs.file-max',nofile,'max_no_files')

	def increaseUserInstances(self,count:int = 1):
		self.current_user_instances += count
		if self.max_user_instances <= self.current_user_instances:
			self.increaseInotifyMaxUserInstances(self.max_user_instances * 2)
		return self.current_user_instances
	
	def decreaseUserInstances(self,count:int = 1):
		self.current_user_instances -= count
		return self.current_user_instances
	
	def increaseUserWatches(self,count:int = 1):
		self.current_user_watches += count
		if self.max_user_watches <= self.current_user_watches:
			self.increaseInotifyMaxUserWatches(self.max_user_watches * 2)
		return self.current_user_watches
	
	def decreaseUserWatches(self,count:int = 1):
		self.current_user_watches -= count
		return self.current_user_watches


In [19]:
irm = inotify_resource_manager()
irm.debug = True

In [20]:
dict(irm)

{'current_user_instances': 0,
 'current_user_watches': 0,
 'can_set_max_queued_events': True,
 'can_set_max_user_instances': True,
 'can_set_max_user_watches': True,
 'can_set_max_no_files': True,
 'can_set_max_no_files_user': True,
 'debug': True,
 'max_queued_events': 16384,
 'max_user_instances': 128,
 'max_user_watches': 483126,
 'max_no_files': 9223372036854775807,
 'max_no_files_user': 524288}

In [21]:
irm.increaseNoFiles(524288 * 2)

Running command: sysctl fs.file-max
----------------------------------------------------------------------------------------------------

fs.file-max = 9223372036854775807
----------------------------------------------------------------------------------------------------
Process exited with return code 0
Failed to increase open file limit to 1048576


False

In [22]:
def initializeFolderWatchers(inotify_obj:inotify_simple.INotify,monitor_path:str,wdDic:dict,irm:inotify_resource_manager,watch_flags:int,newDirWds = None):
	global DEBUG
	if not os.path.exists(monitor_path):
		return -1
	irm.increaseUserWatches()
	if DEBUG:
		print(f'Adding watch for {monitor_path}')
	parentWatchDescriptor = inotify_obj.add_watch(monitor_path, watch_flags)
	if newDirWds is not None:
		newDirWds.add(parentWatchDescriptor)
	wdDic[parentWatchDescriptor] = monitor_path
	allFolders = get_all_folders(monitor_path)
	if DEBUG:
		print(f'Adding watch for {len(allFolders)} sub folders')
	irm.increaseUserWatches(len(allFolders))
	for folder in allFolders:
		childWd = inotify_obj.add_watch(folder, watch_flags)
		if newDirWds is not None:
			newDirWds.add(childWd)
		wdDic[childWd] = folder
	return parentWatchDescriptor


In [23]:
# X		ACCESS = 0x00000001  #: File was accessed
# Y	MODIFY = 0x00000002  #: File was modified
# Y	ATTRIB = 0x00000004  #: Metadata changed
# X		CLOSE_WRITE = 0x00000008  #: Writable file was closed
# X		CLOSE_NOWRITE = 0x00000010  #: Unwritable file closed
# X		OPEN = 0x00000020  #: File was opened
# Y	MOVED_FROM = 0x00000040  #: File was moved from X
# Y	MOVED_TO = 0x00000080  #: File was moved to Y
# Y	CREATE = 0x00000100  #: Subfile was created
# Y	DELETE = 0x00000200  #: Subfile was deleted
# Y	DELETE_SELF = 0x00000400  #: Self was deleted
# Y	MOVE_SELF = 0x00000800  #: Self was moved

# Y	UNMOUNT = 0x00002000  #: Backing fs was unmounted
# Y	Q_OVERFLOW = 0x00004000  #: Event queue overflowed
# X		IGNORED = 0x00008000  #: File was ignored

# X		ONLYDIR = 0x01000000  #: only watch the path if it is a directory
# X		DONT_FOLLOW = 0x02000000  #: don't follow a sym link
# X		EXCL_UNLINK = 0x04000000  #: exclude events on unlinked objects
# X		MASK_ADD = 0x20000000  #: add to the mask of an already existing watch
# Y	ISDIR = 0x40000000  #: event occurred against dir
# X		ONESHOT = 0x80000000  #: only send event once

In [24]:
to_process = deque()
to_process_flag = threading.Event()
monitor_path = 'inotify_test'

In [25]:
class bidict(dict):
	# Credit: https://stackoverflow.com/users/1422096/basj
	# https://stackoverflow.com/questions/3318625/how-to-implement-an-efficient-bidirectional-hash-table
	def __init__(self, *args, **kwargs):
		super(bidict, self).__init__(*args, **kwargs)
		self.inverse = {}
		for key, value in self.items():
			self.inverse.setdefault(value, []).append(key) 

	def __setitem__(self, key, value):
		if key in self:
			self.inverse[self[key]].remove(key) 
		super(bidict, self).__setitem__(key, value)
		self.inverse.setdefault(value, []).append(key)        

	def __delitem__(self, key):
		self.inverse.setdefault(self[key], []).remove(key)
		if self[key] in self.inverse and not self.inverse[self[key]]: 
			del self.inverse[self[key]]
		super(bidict, self).__delitem__(key)

In [26]:
def watchFolder(monitor_path:str,to_process:deque,to_process_flag:threading.Event,irm:inotify_resource_manager,discard_new_recursive_folder_events_until_read:bool = True,min_snapshot_delay_seconds:int=DEFAULT_SNAPSHOT_DELAY):
	global COOKIE_DICT_MAX_SIZE
	global tl
	global DEBUG
	flags = inotify_simple.flags
	watch_flags = flags.MODIFY | flags.ATTRIB | flags.MOVED_FROM | flags.MOVED_TO | flags.CREATE | flags.DELETE | flags.DELETE_SELF | flags.MOVE_SELF | flags.UNMOUNT | flags.Q_OVERFLOW
	selfDestructMask = flags.UNMOUNT | flags.DELETE_SELF | flags.MOVE_SELF
	cookieDic = OrderedDict()
	wdDic = bidict()
	pendingMoveFromEvents = OrderedDict()
	newDirWDs = set()
	processPending = False
	irm.increaseUserInstances()
	with inotify_simple.INotify() as inotify_obj:
		mainWatchDescriptor = initializeFolderWatchers(inotify_obj,monitor_path,wdDic,irm,watch_flags)
		if mainWatchDescriptor == -1:
			tl.teeerror(f'Failed to initialize folder watchers for {monitor_path}')
			return False
		while GREEN_LIGHT.is_set():
			events =  inotify_obj.read(timeout=min_snapshot_delay_seconds * 1000)
			if not events:
				# this means timeout have elapsed, we can signal the backuper to process
				if to_process:
					if DEBUG:
						tl.teeprint('Timeout elapsed, signaling backuper to process')
					to_process_flag.set()
					processPending = True
				continue
			if processPending and not to_process_flag.is_set():
				# this means the to_process was just processed
				newDirWDs.clear()
				processPending = False
			if DEBUG:
				tl.teeok(f'Events detected: {len(events)}')
				decodedEvents = [[event.wd, flags.from_mask(event.mask),event.cookie, event.name] for event in events]
				print(TSVZ.pretty_format_table(decodedEvents,header=['wd', 'mask','cookie','name']))
			monTime = time.monotonic()
			for event in events:
				if event.mask & flags.Q_OVERFLOW:
					tl.teeerror('Queue overflow, event maybe lost')
					tl.teeprint('Attempting to increase inotify max_queued_events')
					irm.increaseInotifyMaxQueueEvents(irm.max_queued_events * 2)
					continue
				isDir = True if event.mask & flags.ISDIR else False
				if event.mask & selfDestructMask:
					if event.wd == mainWatchDescriptor:
						# if UNMOUNT or IGNORED or DELETE_SELF, MOVE_SELF, return false
						if DEBUG:
							tl.teeprint('Main dir self destruct event detected, terminating')
						irm.decreaseUserWatches(len(wdDic))
						irm.decreaseUserInstances()
						to_process.append(ChangeEvent(monTime,isDir,'self_destruct',monitor_path,None))
						return False
					elif event.mask & flags.DELETE_SELF:
						if DEBUG:
							tl.teeprint(f'Sub dir delete self event detected, removing watch for {wdDic.get(event.wd,None)}')
						wdDic.pop(event.wd, None)
						irm.decreaseUserWatches()
						continue
					elif event.mask & flags.UNMOUNT:
						if event.wd in wdDic:
							if DEBUG:
								tl.teeprint(f'Sub dir unmount event detected, resetting watch for {wdDic.get(event.wd,None)}')
							wdDic.pop(event.wd, None)
							wdDic[inotify_obj.add_watch(wdDic[event.wd], watch_flags)] = wdDic[event.wd]
						else:
							if DEBUG:
								tl.teeerror(f'Sub dir unmount event detected, but watch not in wdDic found for {event.wd}')
						continue
					elif event.mask & flags.MOVE_SELF:
						if DEBUG:
							tl.teeprint('Sub dir move self event detected, skipping')
						continue
				if not event.name:
					if DEBUG:
						tl.teeprint('Event name not found, ignoring event')
					continue
				if event.wd not in wdDic:
					if DEBUG:
						tl.teeerror(f'Watch descriptor {event.wd} not found in wdDic {wdDic}, ignoring event')
					continue
				if discard_new_recursive_folder_events_until_read and event.wd in newDirWDs:
					if DEBUG:
						tl.teeprint(f'Discarding event for new recursive folder {wdDic[event.wd]}')
					continue
				eventPath = os.path.join(wdDic[event.wd],event.name)
				if event.mask & flags.CREATE:
					if DEBUG:
						tl.teeprint(f'Create event detected for {eventPath}')
					if isDir:
						initializeFolderWatchers(inotify_obj,eventPath,wdDic,irm,watch_flags,newDirWDs)
					to_process.append(ChangeEvent(monTime,isDir,'create',eventPath,None))
				elif event.mask & flags.DELETE:
					if DEBUG:
						tl.teeprint(f'Delete event detected for {eventPath}')
					# if isDir:
					# 	if DEBUG:
					# 		tl.teeprint(f'Deleting wd {event.wd} for {eventPath}')
					# 	wdDic.pop(event.wd, None)
					# 	try:
					# 		inotify_obj.rm_watch(event.wd)
					# 	except:
					# 		if DEBUG:
					# 			tl.teeerror(f'Failed to remove watch for {event.wd} (already removed?)')
					# 	irm.decreaseUserWatches()
					to_process.append(ChangeEvent(monTime,isDir,'delete',eventPath,None))
				elif event.mask & flags.ATTRIB:
					if DEBUG:
						tl.teeprint(f'Attrib event detected for {eventPath}')
					to_process.append(ChangeEvent(monTime,isDir,'attrib',eventPath,None))
				elif event.mask & flags.MOVED_FROM:
					if DEBUG:
						tl.teeprint(f'Move from event detected for {eventPath}')
					cookieDic[event.cookie] = COOKIE_VALUE(event.wd,eventPath,isDir)
					while len(cookieDic) > COOKIE_DICT_MAX_SIZE:
						if DEBUG:
							tl.teeprint('Cookie dictionary is full, popping oldest item')
						_,cookie_value = cookieDic.popitem(last=False)
						# treat the move as a delete
						# if cookie_value.isDir:
						# 	if DEBUG:
						# 		tl.teeprint(f'Deleting wd {cookie_value.wd} for {cookie_value.path}')
						# 	wdDic.pop(cookie_value.wd, None)
						# 	try:
						# 		inotify_obj.rm_watch(cookie_value.wd)
						# 	except:
						# 		if DEBUG:
						# 			tl.teeerror(f'Failed to remove watch for {cookie_value.wd} (already removed?)')
						# 	irm.decreaseUserWatches()
						if DEBUG:
							tl.teeprint(f'Adding delete event for {cookie_value.path}')
						to_process.append(ChangeEvent(monTime,isDir,'delete',cookie_value.path,None))
					#to_process.append(ChangeEvent(monTime,isDir,'move',eventPath,None))
					pendingMoveFromEvents[event.cookie] = ChangeEvent(monTime,isDir,'delete',eventPath,None)
				elif event.mask & flags.MOVED_TO:
					if DEBUG:
						tl.teeprint(f'Move to event detected for {eventPath}')
					if event.cookie in cookieDic:
						if DEBUG:
							tl.teeprint(f'Cookie found for {event.cookie}')
							tl.teeprint(f'Adding move event for {cookieDic[event.cookie].path} to {eventPath}')
						cookie_value = cookieDic.pop(event.cookie)
						pendingMoveFromEvents.pop(event.cookie,None)
						to_process.append(ChangeEvent(monTime,isDir,'move',eventPath,cookie_value.path))
						if isDir:
							# we need to update wdDic to reflect the new path
							if cookie_value.path in wdDic.inverse:
								if DEBUG:
									tl.teeprint(f'Updating all values of wdDic {wdDic.inverse[cookie_value.path]} for {cookie_value.path} to {eventPath}')
								wds = wdDic.inverse[cookie_value.path]
								for wd in wds:
									wdDic[wd] = eventPath
					else:
						if DEBUG:
							tl.teeprint(f'Cookie not found for {event.cookie}')
							tl.teeprint(f'Treating move as create event for {eventPath}')
						if isDir:
							initializeFolderWatchers(inotify_obj,eventPath,wdDic,irm,watch_flags,newDirWDs)
						to_process.append(ChangeEvent(monTime,isDir,'create',eventPath,None))
				elif event.mask & flags.MODIFY:
					if DEBUG:
						tl.teeprint(f'Modify event detected for {eventPath}')
					to_process.append(ChangeEvent(monTime,isDir,'modify',eventPath,None))
				elif DEBUG:
					tl.info(f'Unprocessed event {event}')
			if pendingMoveFromEvents:
				if DEBUG:
					tl.teeprint(f'Adding unmatched pending move_from events as deletes {pendingMoveFromEvents.keys()}')
				to_process.extend(pendingMoveFromEvents.values())
				pendingMoveFromEvents.clear()
	return True


In [27]:
#watchFolder(monitor_path,to_process,to_process_flag,irm)

In [ ]:
os.chdir('/tmp')
os.system('mkdir -p inotify_test')
to_process.clear()
watchFolderThread = threading.Thread(target=watchFolder,args=(monitor_path,to_process,to_process_flag,irm),daemon=True)
watchFolderThread.start()

Adding watch for inotify_test
Adding watch for 0 sub folders


Events detected: 1
wd | mask                  | cookie | name          
---+-----------------------+--------+---------------
1  | [<flags.CREATE: 256>] | 0      | write_test.txt

Create event detected for inotify_test/write_test.txt
Events detected: 1
wd | mask                | cookie | name          
---+---------------------+--------+---------------
1  | [<flags.MODIFY: 2>] | 0      | write_test.txt

Modify event detected for inotify_test/write_test.txt
Events detected: 1
wd | mask                | cookie | name          
---+---------------------+--------+---------------
1  | [<flags.MODIFY: 2>] | 0      | write_test.txt

Modify event detected for inotify_test/write_test.txt
Events detected: 2
wd | mask                     | cookie | name               
---+--------------------------+--------+--------------------
1  | [<flags.MOVED_FROM: 64>] | 5891   | write_test.txt     
1  | [<flags.MOVED_TO: 128>]  | 5891   | move_test_trick.txt

Move from event detected for inotify_test/write_t

In [29]:
def testWatcherOperation(testCMD:list,to_process:deque,to_process_flag:threading.Event):
	multiCMD.run_command(testCMD)
	# to_process_flag.wait(1)
	# print(to_process)
	# to_process.clear()
	# to_process_flag.clear()


In [30]:
os.chdir('/tmp')
commands = [
	'echo hello > inotify_test/write_test.txt',
	'echo world >> inotify_test/write_test.txt',
	'mv inotify_test/write_test.txt inotify_test/move_test_trick.txt',
	'mv inotify_test/move_test_trick.txt outside.txt',
	'echo hello > outside2.txt',
	'mv outside2.txt inotify_test/move_test_trick.txt',
	'touch inotify_test/touch_test.txt',
	'mv inotify_test/touch_test.txt outside3.txt ; mv outside3.txt inotify_test/same_file_move_out_in.txt',
	'rm inotify_test/same_file_move_out_in.txt',
	'mkdir -p inotify_test/newDirTest',
	'mv inotify_test/newDirTest inotify_test/dirMoveTest',
	'touch inotify_test/dirMoveTest/subfolder.txt',
	'echo hello > inotify_test/dirMoveTest/write_to_sub_folder.txt',
	'mv inotify_test/dirMoveTest/write_to_sub_folder.txt inotify_test/dirMoveTest/move_with_in_sub_folder.txt',
	'mv inotify_test/dirMoveTest/move_with_in_sub_folder.txt inotify_test/move_out_from_sub_folder.txt',
	'touch inotify_test/new_file.txt',
	'mv inotify_test/new_file.txt inotify_test/dirMoveTest/move_into_sub_folder.txt',
	'touch inotify_test/dirMoveTest/test_temp.txt',
	'mkdir -p inotify_test/test/testrecursivefolder/nestedFolder',
	'touch inotify_test/test/testrecursivefolder/nestedFolder/nestedFile.txt',
	'rm -rf inotify_test',
]
os.system('mkdir -p inotify_test')
for cmd in commands:
	testWatcherOperation(['bash','-c',cmd],to_process,to_process_flag)

Running command: bash -c echo hello > inotify_test/write_test.txt
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Process exited with return code 0
Running command: bash -c echo world >> inotify_test/write_test.txt
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Process exited with return code 0
Running command: bash -c mv inotify_test/write_test.txt inotify_test/move_test_trick.txt
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Process exited with return code 0
Running command: bash -c mv inotify_test/move_test_trick.txt outsid

In [31]:
watchFolderThread.is_alive()

True

In [32]:
import importlib
importlib.reload(TSVZ)

<module 'TSVZ' from '/var/home/kes/Documents/reverberator/.venv/lib64/python3.13/site-packages/TSVZ.py'>

In [33]:
print(TSVZ.pretty_format_table(to_process,header = CHANGED_EVENT_HEADER))

monotonic_time  | is_dir | event         | path                                      | moved_from                 
----------------+--------+---------------+-------------------------------------------+----------------------------
12193.22433354  | False  | create        | inotify_test/write_test.txt               | None                       
12193.224396768 | False  | modify        | inotify_test/write_test.txt               | None                       
12193.225582703 | False  | modify        | inotify_test/write_test.txt               | None                       
12193.227650914 | False  | move          | inotify_test/move_test_trick.txt          | inotify_test/write_test.txt
12193.230129264 | False  | delete        | inotify_test/move_test_trick.txt          | None                       
12193.233725082 | False  | create        | inotify_test/move_test_trick.txt          | None                       
12193.236717987 | False  | create        | inotify_test/touch_test.txt          

In [34]:

def is_subpath(child, parent):
    child = os.path.realpath(child)
    parent = os.path.realpath(parent)
    common = os.path.commonpath([child, parent])
    return common == parent

In [35]:
def change_events_to_backup_entries(change_events:deque) -> OrderedDict:
	# possibile events are create, delete, attrib, move, modify
	# CHANGED_EVENT_HEADER = ['monotonic_time', 'is_dir', 'event', 'path','moved_from']
	# BACKUP_ENTRY_VALUES_HEADER = ['iso_time','event','source_path']
	# collapse the change events
	# for a single file, only these events will be kept:
	# 	modify, attrib, move, delete ( create will be treated as modify as we will be copying the entire file )
	# 	modify will be exclusive to file
	# for a single directory, only the following kinds of events will be kept:
	# 	create, delete, attrib, move ( modify should not be possible but just in case, it will be treated as create )
	# 	folder create will recursively copy all its contents 
	# 	create should be exclusive to folder
	# Only move what with a moved_from will be kept, all moves without moved_from will be treated as delete
	# will get the iso_time using (datetime.datetime.now() - datetime.timedelta(seconds=time.monotonic() - monTime)).isoformat())
	# will indicate if a path is a dir by making sure it ends with /
	# if it is a self_destruct event, we still keep it in 
	global DEBUG
	global tl
	if DEBUG:
		tl.teeprint('Converting the following change events to backup entries')
		tl.printTable(change_events,header = CHANGED_EVENT_HEADER)
	backup_entries = OrderedDict()
	newDirectories = set()
	pendingMoveSourceParents = set()
	moveSourceToDestDict = {}
	iso_time = ''
	for changeEvent in reversed(change_events):
		# get the absolute path
		to_add = True
		abs_path = os.path.abspath(changeEvent.path)
		if changeEvent.moved_from:
			abs_moved_from = os.path.abspath(changeEvent.moved_from)
			if changeEvent.is_dir:
				abs_moved_from += '/'
		else:
			abs_moved_from = None
		event = changeEvent.event
		if changeEvent.is_dir:
			abs_path += '/'
		if DEBUG:
			tl.teeprint('-'*80)
			tl.teeprint(f'Processing {event} on {abs_path} with source {abs_moved_from}')
		# pre handle move events
		if event == 'move':
			if abs_moved_from:
				pendingMoveSourceParents.add(abs_moved_from)
				if abs_path in moveSourceToDestDict:
					# chain move, we will modify the chronologically later move event's source to this event's source
					finialTarget = moveSourceToDestDict[abs_path]
					moveSourceToDestDict[abs_moved_from] = finialTarget
					del moveSourceToDestDict[abs_path]
					if finialTarget in backup_entries:
						backup_entries[finialTarget] = BackupEntryValues(backup_entries[finialTarget].iso_time,backup_entries[finialTarget].event,abs_moved_from)
						if DEBUG:
							if backup_entries[finialTarget].event != 'move':
								tl.teeprint(f'  Chain move (now {backup_entries[finialTarget].event}) from {abs_moved_from} through {abs_path} to {finialTarget}')
							else:
								tl.teeprint(f'  Chain move from {abs_moved_from} through {abs_path} to {finialTarget}')
					else:
						# this means the later move event was deleted, no need to update it
						if DEBUG:
							tl.teeprint(f'  Already removed chain move event from {abs_moved_from} through {abs_path} to {finialTarget}')
						if abs_path in backup_entries:
							if DEBUG:
								tl.teeprint(f'    Removing {backup_entries[abs_path].event} event on {abs_path}')
							del backup_entries[abs_path]
					to_add = False
				else:
					moveSourceToDestDict[abs_moved_from] = abs_path
			else:
				if abs_path in pendingMoveSourceParents:
					# we have handled this event's children, so we can ignore this event
					pendingMoveSourceParents.remove(abs_path)
					if DEBUG:
						tl.teeprint(f'Skipping child move event {abs_path}')
					to_add = False
				else:
					# virgin move, treat as create
					if DEBUG:
						tl.teeprint(f'Virgin move event {abs_path}')
					event = 'create'
		elif abs_path in moveSourceToDestDict:
			# This means the path was chronologically later moved.
			# the event can be 'create', 'modify', 'attrib', 'delete' at this point
			# in all cases, as we cannot simply perform a recursive link as a move in the backup, ( need to backup source data )
			# we will break the move event into a delete of source and a {event} of dest
			# here, escalate the current event to the chronologically later move
			if event == 'delete':
				# if the path was chronologically later moved from, then it must exist, thus this delete is likely invalid
				if DEBUG:
					tl.teeprint(f'Delete event detected immediately before a move with {abs_path} as source. Ignoring delete')
				to_add = False
			elif event == 'attrib':
				# escalate to check file content also as it is not possible to sync attrib only to a symlink 
				event = 'modify'
			moveDest = moveSourceToDestDict[abs_path]
			if DEBUG:
				tl.teeprint(f'  Seperating move event from {abs_path} to {moveDest}')
			if moveDest in backup_entries:
				if event == 'create':
					to_add = False
					escalated_event = 'create'
				else:
					escalated_event = event
					event = 'delete'
				if DEBUG:
					tl.teeprint(f'    Escalating {backup_entries[moveDest].event} to {escalated_event} at {backup_entries[moveDest].iso_time} on {moveDest}')
				backup_entries[moveDest] = BackupEntryValues(backup_entries[moveDest].iso_time,escalated_event,abs_path)
			else:
				if DEBUG:
					tl.teeprint(f'    Already removed event on {moveDest}, skipping escalating')
				if abs_path in backup_entries:
					if DEBUG:
						tl.teeprint(f'    Removing {backup_entries[abs_path].event} event on {abs_path}')
					del backup_entries[abs_path]
				continue
			
		# handle repeated events on the same path
		if abs_path in backup_entries:
			if DEBUG:
				tl.teeprint(f'  {abs_path} already exists in backup entries')
			# handle repeated events on the same path
			later_event_on_path = backup_entries[abs_path].event
			if event == 'create':
				if later_event_on_path == 'delete':
					# if the path was new and then chronologically later deleted, then we ignore both events
					if DEBUG:
						tl.teeprint(f'Skipping event as Creating {abs_path} that is later deleted.')
						tl.teeprint(f'  Removing delete event {abs_path}:{backup_entries[abs_path]} as it was created in this iteration')
					del backup_entries[abs_path]
					to_add = False
				elif later_event_on_path in ('modify','attrib'):
					# this is a new file that is later modified. We need to remove the later entry add this create instead.
					if DEBUG:
						tl.teeprint(f'Escalating later {later_event_on_path} event to create on {abs_path}')
					del backup_entries[abs_path]
				elif later_event_on_path == 'move':
					# this is a new file that later got moved to. we can safely ignore this event
					if DEBUG:
						tl.teeprint(f'Skipping event as Creating {abs_path} that is later moved to.')
					to_add = False
			elif later_event_on_path in ('delete','modify','move'):
				# if the path is chronologically later deleted, then we simply do not need to perform backups
				# if the path is chronologically later modified, because the current event can be modify / attrib / move / delete , in all cases we will ignore this event
				# if the path is chronologically later moved to, meaning it was later overwriiten moved to. We do not care what it had been done to before then.
				# in this version of reverberator, create is a complete file tree copy, so if it is chronologically later "created", all previoud events no longer matter.
				# in the case of move, because we are skipping this event without adding it to the move sources,
				# the next chronological move event of the source will be treated as a delete
				if event == 'move':
					# we also need to add a event to remove the source 
					if DEBUG:
						tl.teeprint(f'Moving to {abs_path} that is later {later_event_on_path}d.')
					if later_event_on_path == 'delete':
						if DEBUG:
							tl.teeprint(f'  Removing delete event {abs_path}:{backup_entries[abs_path]} as it was created by move in this iteration')
						del backup_entries[abs_path]
					abs_path = abs_moved_from
					abs_moved_from = None
					event = 'delete'
				else:
					if DEBUG:
						tl.teeprint(f'Skipping {event} on {abs_path}')
					to_add = False
			elif later_event_on_path == 'attrib':
				if event in ('attrib','delete'):
					# delete: the path was chronologically later attributed, so it must exist later, thus we ignore the delete
					if DEBUG:
						tl.teeprint(f'Skipping dup {event} on {abs_path}')
					to_add = False
				if event in ('modify', 'create', 'move'):
					# we need to escalate the chronologically later event to modify / create / move as attrib only copies the metadata
					if DEBUG:
						tl.teeprint(f'Escalating attrib event to {event} on {abs_path}')
					del backup_entries[abs_path]
		if to_add:
			if not iso_time:
				iso_time = (datetime.datetime.now() - datetime.timedelta(seconds=time.monotonic() - changeEvent.monotonic_time)).isoformat()
			if DEBUG:
				tl.teeprint(f'Adding {event} on {abs_path} at {iso_time} with source {abs_moved_from}')
			backup_entries[abs_path] = BackupEntryValues(iso_time,event,abs_moved_from)
			backup_entries.move_to_end(abs_path,last=False)
	if DEBUG:
		tl.teeprint('Converted backup entries')
		tl.printTable(backup_entries,header = ['path'] + BACKUP_ENTRY_VALUES_HEADER)
	return backup_entries

In [36]:
backupEntries = change_events_to_backup_entries(to_process.copy())

Converting the following change events to backup entries
monotonic_time  | is_dir | event         | path                                      | moved_from                 
----------------+--------+---------------+-------------------------------------------+----------------------------
12193.22433354  | False  | create        | inotify_test/write_test.txt               | None                       
12193.224396768 | False  | modify        | inotify_test/write_test.txt               | None                       
12193.225582703 | False  | modify        | inotify_test/write_test.txt               | None                       
12193.227650914 | False  | move          | inotify_test/move_test_trick.txt          | inotify_test/write_test.txt
12193.230129264 | False  | delete        | inotify_test/move_test_trick.txt          | None                       
12193.233725082 | False  | create        | inotify_test/move_test_trick.txt          | None                       
12193.236717987 | False

In [37]:
list(to_process)[:20]

[ChangeEvent(monotonic_time=12193.22433354, is_dir=False, event='create', path='inotify_test/write_test.txt', moved_from=None),
 ChangeEvent(monotonic_time=12193.224396768, is_dir=False, event='modify', path='inotify_test/write_test.txt', moved_from=None),
 ChangeEvent(monotonic_time=12193.225582703, is_dir=False, event='modify', path='inotify_test/write_test.txt', moved_from=None),
 ChangeEvent(monotonic_time=12193.227650914, is_dir=False, event='move', path='inotify_test/move_test_trick.txt', moved_from='inotify_test/write_test.txt'),
 ChangeEvent(monotonic_time=12193.230129264, is_dir=False, event='delete', path='inotify_test/move_test_trick.txt', moved_from=None),
 ChangeEvent(monotonic_time=12193.233725082, is_dir=False, event='create', path='inotify_test/move_test_trick.txt', moved_from=None),
 ChangeEvent(monotonic_time=12193.236717987, is_dir=False, event='create', path='inotify_test/touch_test.txt', moved_from=None),
 ChangeEvent(monotonic_time=12193.236717987, is_dir=False, e

In [38]:
backupEntries = change_events_to_backup_entries(deque(list(to_process)[:20]))

Converting the following change events to backup entries
monotonic_time  | is_dir | event  | path                                      | moved_from                 
----------------+--------+--------+-------------------------------------------+----------------------------
12193.22433354  | False  | create | inotify_test/write_test.txt               | None                       
12193.224396768 | False  | modify | inotify_test/write_test.txt               | None                       
12193.225582703 | False  | modify | inotify_test/write_test.txt               | None                       
12193.227650914 | False  | move   | inotify_test/move_test_trick.txt          | inotify_test/write_test.txt
12193.230129264 | False  | delete | inotify_test/move_test_trick.txt          | None                       
12193.233725082 | False  | create | inotify_test/move_test_trick.txt          | None                       
12193.236717987 | False  | create | inotify_test/touch_test.txt               |

In [39]:
backupEntries = change_events_to_backup_entries(deque(list(to_process)[:18]))

Converting the following change events to backup entries
monotonic_time  | is_dir | event  | path                                      | moved_from                 
----------------+--------+--------+-------------------------------------------+----------------------------
12193.22433354  | False  | create | inotify_test/write_test.txt               | None                       
12193.224396768 | False  | modify | inotify_test/write_test.txt               | None                       
12193.225582703 | False  | modify | inotify_test/write_test.txt               | None                       
12193.227650914 | False  | move   | inotify_test/move_test_trick.txt          | inotify_test/write_test.txt
12193.230129264 | False  | delete | inotify_test/move_test_trick.txt          | None                       
12193.233725082 | False  | create | inotify_test/move_test_trick.txt          | None                       
12193.236717987 | False  | create | inotify_test/touch_test.txt               |

In [40]:
backupEntries

OrderedDict([('/tmp/inotify_test/dirMoveTest/',
              BackupEntryValues(iso_time='2025-03-31T19:25:56.964076', event='create', source_path='/tmp/inotify_test/newDirTest/')),
             ('/tmp/inotify_test/move_out_from_sub_folder.txt',
              BackupEntryValues(iso_time='2025-03-31T19:25:56.964076', event='create', source_path=None)),
             ('/tmp/inotify_test/test/',
              BackupEntryValues(iso_time='2025-03-31T19:25:56.964076', event='create', source_path=None))])

In [41]:
print(TSVZ.pretty_format_table(backupEntries,header = ['path'] + BACKUP_ENTRY_VALUES_HEADER))

path                                           | iso_time                   | event  | source_path                  
-----------------------------------------------+----------------------------+--------+------------------------------
/tmp/inotify_test/dirMoveTest/                 | 2025-03-31T19:25:56.964076 | create | /tmp/inotify_test/newDirTest/
/tmp/inotify_test/move_out_from_sub_folder.txt | 2025-03-31T19:25:56.964076 | create | None                         
/tmp/inotify_test/test/                        | 2025-03-31T19:25:56.964076 | create | None                         



In [42]:
os.chdir('/tmp')
os.system('mkdir -p inotify_test')

commands = [
	'mkdir -p inotify_test/test/testrecursivefolder/nestedFolder',
	'echo hello > inotify_test/write_test.txt',
	'mkdir -p inotify_test/newDirTest',
]
for cmd in commands:
	testWatcherOperation(['bash','-c',cmd],to_process,to_process_flag)
to_process.clear()
watchFolderThread = threading.Thread(target=watchFolder,args=(monitor_path,to_process,to_process_flag,irm),daemon=True)
watchFolderThread.start()
commands = [
	'echo world >> inotify_test/write_test.txt',
	'mv inotify_test/write_test.txt inotify_test/move_test_trick.txt',
	'mv inotify_test/move_test_trick.txt outside.txt',
	'echo hello > outside2.txt',
	'mv outside2.txt inotify_test/move_test_trick.txt',
	'touch inotify_test/touch_test.txt',
	'mv inotify_test/touch_test.txt outside3.txt ; mv outside3.txt inotify_test/same_file_move_out_in.txt',
	'echo what >> inotify_test/newDirTest/before_move.txt',
	'mv inotify_test/newDirTest inotify_test/dirMoveTest',
	'touch inotify_test/dirMoveTest/subfolder.txt',
]
for cmd in commands:
	testWatcherOperation(['bash','-c',cmd],to_process,to_process_flag)
to_process_flag.clear()
commands = [
	'echo hello > inotify_test/dirMoveTest/write_to_sub_folder.txt',
	'mv inotify_test/dirMoveTest/write_to_sub_folder.txt inotify_test/dirMoveTest/move_with_in_sub_folder.txt',
	'mv inotify_test/dirMoveTest/move_with_in_sub_folder.txt inotify_test/move_out_from_sub_folder.txt ; mv inotify_test/move_out_from_sub_folder.txt inotify_test/test/testrecursivefolder/nestedFolder/nestedFile.txt',
	'touch inotify_test/new_file.txt',
	'mv inotify_test/new_file.txt inotify_test/dirMoveTest/move_into_sub_folder.txt',
	'touch inotify_test/dirMoveTest/test_temp.txt',
	'touch inotify_test/test/testrecursivefolder/nestedFolder/nestedFile.txt',
	# 'rm -rf inotify_test',
]
for cmd in commands:
	testWatcherOperation(['bash','-c',cmd],to_process,to_process_flag)

Running command: bash -c mkdir -p inotify_test/test/testrecursivefolder/nestedFolder
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Process exited with return code 0
Running command: bash -c echo hello > inotify_test/write_test.txt
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Process exited with return code 0
Running command: bash -c mkdir -p inotify_test/newDirTest
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Process exited with return code 0
Running command: bash -c echo world >> inotify_test/write_test.txt
-------------

Events detected: 1
wd | mask                    | cookie | name               
---+-------------------------+--------+--------------------
1  | [<flags.MOVED_TO: 128>] | 5902   | move_test_trick.txt

Move to event detected for inotify_test/move_test_trick.txt
Cookie not found for 5902
Treating move as create event for inotify_test/move_test_trick.txt

----------------------------------------------------------------------------------------------------
Process exited with return code 0
Running command: bash -c touch inotify_test/touch_test.txt
----------------------------------------------------------------------------------------------------
Events detected: 2
wd | mask                  | cookie | name          
---+-----------------------+--------+---------------
1  | [<flags.CREATE: 256>] | 0      | touch_test.txt
1  | [<flags.ATTRIB: 4>]   | 0      | touch_test.txt

Create event detected for inotify_test/touch_test.txt
Attrib event detected for inotify_test/touch_test.txt

----------

Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elapsed, signaling backuper to process
Timeout elaps

In [43]:
backup_entries = change_events_to_backup_entries(deque(list(to_process)[:23]))

Converting the following change events to backup entries
monotonic_time  | is_dir | event  | path                                                              | moved_from                                          
----------------+--------+--------+-------------------------------------------------------------------+-----------------------------------------------------
12193.349244238 | False  | move   | inotify_test/move_test_trick.txt                                  | inotify_test/write_test.txt                         
12193.351501704 | False  | delete | inotify_test/move_test_trick.txt                                  | None                                                
12193.356490495 | False  | create | inotify_test/move_test_trick.txt                                  | None                                                
12193.358393585 | False  | create | inotify_test/touch_test.txt                                       | None                                                
1

In [44]:
backup_entries

OrderedDict([('/tmp/inotify_test/write_test.txt',
              BackupEntryValues(iso_time='2025-03-31T19:25:57.086556', event='delete', source_path=None)),
             ('/tmp/inotify_test/same_file_move_out_in.txt',
              BackupEntryValues(iso_time='2025-03-31T19:25:57.086556', event='create', source_path=None)),
             ('/tmp/inotify_test/newDirTest/before_move.txt',
              BackupEntryValues(iso_time='2025-03-31T19:25:57.086556', event='create', source_path=None)),
             ('/tmp/inotify_test/dirMoveTest/',
              BackupEntryValues(iso_time='2025-03-31T19:25:57.086556', event='move', source_path='/tmp/inotify_test/newDirTest/')),
             ('/tmp/inotify_test/dirMoveTest/subfolder.txt',
              BackupEntryValues(iso_time='2025-03-31T19:25:57.086556', event='create', source_path=None)),
             ('/tmp/inotify_test/test/testrecursivefolder/nestedFolder/nestedFile.txt',
              BackupEntryValues(iso_time='2025-03-31T19:25:57.086556',

In [45]:
import json
print(json.dumps(backup_entries,indent=2))

{
  "/tmp/inotify_test/write_test.txt": [
    "2025-03-31T19:25:57.086556",
    "delete",
    null
  ],
  "/tmp/inotify_test/same_file_move_out_in.txt": [
    "2025-03-31T19:25:57.086556",
    "create",
    null
  ],
  "/tmp/inotify_test/newDirTest/before_move.txt": [
    "2025-03-31T19:25:57.086556",
    "create",
    null
  ],
  "/tmp/inotify_test/dirMoveTest/": [
    "2025-03-31T19:25:57.086556",
    "move",
    "/tmp/inotify_test/newDirTest/"
  ],
  "/tmp/inotify_test/dirMoveTest/subfolder.txt": [
    "2025-03-31T19:25:57.086556",
    "create",
    null
  ],
  "/tmp/inotify_test/test/testrecursivefolder/nestedFolder/nestedFile.txt": [
    "2025-03-31T19:25:57.086556",
    "create",
    "/tmp/inotify_test/dirMoveTest/write_to_sub_folder.txt"
  ],
  "/tmp/inotify_test/dirMoveTest/move_into_sub_folder.txt": [
    "2025-03-31T19:25:57.086556",
    "create",
    "/tmp/inotify_test/new_file.txt"
  ],
  "/tmp/inotify_test/dirMoveTest/test_temp.txt": [
    "2025-03-31T19:25:57.086556",
   

In [46]:
BACKUP_JOURNAL_HEADER = ['iso_time','event','at_path','source_path']

In [47]:
def log_events_to_journal(backup_entries:dict,journal_path:str):
	# this function will log the journal of the changes
	# the journal will be a nsv file with the following fields:
	# monotonic_time, event, type, path
	# this will be used to recover the changes in the case of a crash
	lines = [[entry.iso_time,entry.event,path,entry.source_path] for path,entry in backup_entries.items()]
	TSVZ.appendLinesTabularFile(journal_path,lines,header = BACKUP_JOURNAL_HEADER,createIfNotExist = True,verifyHeader = True,strict=False)

In [48]:
import datetime
datetime.datetime.now().astimezone().strftime('%Y-%m-%dT%H-%M-%SZ%z')

'2025-03-31T19-25-57Z-0700'